In [55]:
import requests
from bs4 import BeautifulSoup as bs
import types
import json
import pymysql

owner_id="9273190"
user_url = "https://stackoverflow.com/users/" + owner_id
user_page = bs(requests.get(user_url).text,'html.parser')
    
reputation = user_page.find('div',class_='reputation').text.split(' r')[0]#可能大於一千，就會有,不能直接轉型
reputation = int(reputation.strip().replace(',',''))
print(reputation)

times=0
sql_text = ""
for x in user_page.find_all('a',class_='post-tag'):
    sql_text += x.text + "|"   
    for y in user_page.find_all('span',class_='g-col -number'):
        times+=1
        if(times==1 or times==6 or times==13 or times==22 or times==33 or times==46):#score
            sql_text += y.text + ',' 
        if(times==2 or times==7 or times==14 or times==23 or times==34 or times==47):#post
            sql_text += y.text + " , "
            break
            
print(sql_text)

7
grep|0,1 , bash|0,1 , sed|0,1 , c++|0,1 , compilation|0,1 , header|0,1 , 


In [63]:
answers = int(user_page.find('div',class_='stat answers col-3').text.split('a')[0].strip().replace(',',''))
print(answers)

questions = int(user_page.find('div',class_='stat questions col-3').text.split('q')[0].strip().replace(',',''))
print(questions)

reached = user_page.find('div',class_='stat people-helped col-5').text.split('p')[0].strip().replace('~','').replace(',','')

if(reached[-1] == 'k'):
    reached = float(reached.replace('k',''))
    reached = reached * 1000
elif(reached[-1] == 'm'):
    reached = float(reached.replace('m',''))
    reached = reached * 1000000
else:
    reached = float(reached)
    
print(reached)

#1593077、9273190
profile = user_page.find('ul',class_='list-unstyled')
location = ''
age = ''
views = ''
for x in profile.find_all('li'):
    if(x.find('svg',class_='svg-icon iconLocation') != None):#代表這傢伙有居住地
        location = x.text.strip()
    if(x.find('svg',class_='svg-icon iconHistory') != None):#這傢伙的帳號創建日
        age = x.find('span')['title']
    if(x.find('svg',class_='svg-icon iconEye') != None):
        views = int(x.text.split(' p')[0].strip().replace(',',''))
        
print(location + ' | ' , age , ' | ' , views)

0
2
128.0
 |  2018-01-26 15:44:38Z  |  3


In [61]:
toptags_url = 'https://api.stackexchange.com/2.2/users/' + owner_id + '/top-tags?site=stackoverflow'
user_toptags = bs(requests.get(toptags_url).text,'html.parser')


json_str = json.loads(user_toptags.text)

total_count = 0
total_score = 0
for x in range(len(json_str['items'])):
    total_count = json_str['items'][x]['answer_count'] + json_str['items'][x]['question_count']
    total_score = json_str['items'][x]['answer_score'] + json_str['items'][x]['question_score']
    print('answer_count: ',json_str['items'][x]['answer_count'],'answer_score: ',json_str['items'][x]['answer_score'],
         'question_count: ',json_str['items'][x]['question_count'],'question_score: ',json_str['items'][x]['question_score'],
         'total_count:',total_count,'total_score: ',total_score,'tag_name: ',json_str['items'][x]['tag_name'])

In [67]:
db = pymysql.connect('140.138.77.90','knkn','tp6bjo4u;6','knkn',use_unicode=True, charset="utf8mb4")
cursor = db.cursor()
cursor.execute('INSERT INTO user (user_id,reputation,answers,questions,reached,location,age,views) VALUES('
               + '"' + owner_id + '",' + str(reputation) + ',' + str(answers) + ',' + str(questions) + ',' + str(reached) + ',"' + location + '","' + age + '",' + str(views) + ')')

db.commit()
cursor.close()
db.close()